In [33]:
import numpy as np
import scipy as sp
import pandas as pd
from IPython.display import display, HTML
df1=pd.read_csv("malware_Binary.csv")



In [34]:
(df1.head())

,hash,millisecond,classification,os,state,usage_counter,prio,static_prio,normal_prio,policy,...,nivcsw,min_flt,maj_flt,fs_excl_counter,lock,utime,stime,gtime,cgtime,signal_nvcsw
0,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,0,malware,CentOS,0,0,3069378560,14274,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
1,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,1,malware,Windows,0,0,3069378560,14274,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
2,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,2,malware,Mac,0,0,3069378560,14274,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
3,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,3,malware,Ubuntu,0,0,3069378560,14274,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
4,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,4,malware,Mac,0,0,3069378560,14274,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0


In [35]:
df=df1.loc[:,['classification', 'os', 'usage_counter', 'prio', 'static_prio', 'normal_prio', 'vm_pgoff', 'vm_truncate_count', 'task_size', 'map_count', 'hiwater_rss', 'total_vm', 'shared_vm', 'exec_vm', 'reserved_vm', 'nr_ptes', 'nvcsw', 'nivcsw', 'signal_nvcsw']]

In [36]:
df.head()

,classification,os,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw
0,malware,CentOS,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
1,malware,Windows,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
2,malware,Mac,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
3,malware,Ubuntu,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
4,malware,Mac,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0


In [37]:
# Converting os and classification into integer variables
df_dummies=pd.get_dummies(df[['os']], dtype=float)
print(df_dummies.head())

   os_CentOS  os_Debian  os_Mac  os_Ubuntu  os_Windows
0        1.0        0.0     0.0        0.0         0.0
1        0.0        0.0     0.0        0.0         1.0
2        0.0        0.0     1.0        0.0         0.0
3        0.0        0.0     0.0        1.0         0.0
4        0.0        0.0     1.0        0.0         0.0


In [38]:
df = df.join(df_dummies)
df = df.drop('os', axis=1)

In [39]:
df.head()

,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,...,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,malware,0,3069378560,14274,0,0,13173,0,6850,0,...,210,0,341974,0,0,1.0,0.0,0.0,0.0,0.0
1,malware,0,3069378560,14274,0,0,13173,0,6850,0,...,210,0,341974,0,0,0.0,0.0,0.0,0.0,1.0
2,malware,0,3069378560,14274,0,0,13173,0,6850,0,...,210,0,341974,0,0,0.0,0.0,1.0,0.0,0.0
3,malware,0,3069378560,14274,0,0,13173,0,6850,0,...,210,0,341974,0,0,0.0,0.0,0.0,1.0,0.0
4,malware,0,3069378560,14274,0,0,13173,0,6850,0,...,210,0,341974,0,0,0.0,0.0,1.0,0.0,0.0


In [40]:
df=df.drop('os_CentOS',axis=1) #Dropping os_centOS to maintain n-1 columns

In [41]:
for col in df.columns:
    # removing classification as it is the label
    if col != 'classification':
        df[col]=(df[col]-df[col].min())/(df[col].max()-df[col].min())
        # both min and max value of the predictor task_size is zero so dropped.
    # it does not contribute to distance calculation
df = df.drop('task_size', axis= 1)

In [42]:
df.isna().sum()

classification            0
usage_counter        100000
prio                      0
static_prio               0
normal_prio          100000
vm_pgoff             100000
vm_truncate_count         0
map_count                 0
hiwater_rss          100000
total_vm                  0
shared_vm                 0
exec_vm                   0
reserved_vm               0
nr_ptes              100000
nvcsw                     0
nivcsw                    0
signal_nvcsw         100000
os_Debian                 0
os_Mac                    0
os_Ubuntu                 0
os_Windows                0
dtype: int64

In [43]:
df.describe()

,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_Debian,os_Mac,os_Ubuntu,os_Windows
count,0.0,100000.000000,100000.000000,0.0,0.0,100000.000000,100000.000000,0.0,100000.000000,100000.000000,100000.000000,100000.000000,0.0,100000.000000,100000.000000,0.0,100000.000000,100000.000000,100000.000000,100000.000000
mean,NaN,0.499762,0.234841,NaN,NaN,0.321712,0.241567,NaN,0.093546,0.740030,0.343059,0.242872,NaN,0.226876,0.090387,NaN,0.199140,0.199460,0.200680,0.201100
std,NaN,0.287065,0.258006,NaN,NaN,0.186489,0.147887,NaN,0.111189,0.389611,0.214211,0.155259,NaN,0.194690,0.144466,NaN,0.399356,0.399596,0.400511,0.400825
min,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
25%,NaN,0.248016,0.020373,NaN,NaN,0.169110,0.150023,NaN,0.033856,0.250000,0.192308,0.114325,NaN,0.091519,0.002740,NaN,0.000000,0.000000,0.000000,0.000000
50%,NaN,0.492063,0.121509,NaN,NaN,0.317833,0.206165,NaN,0.061654,1.000000,0.336538,0.225895,NaN,0.204049,0.024658,NaN,0.000000,0.000000,0.000000,0.000000
75%,NaN,0.743056,0.458611,NaN,NaN,0.456305,0.316299,NaN,0.115110,1.000000,0.442308,0.336088,NaN,0.298492,0.126027,NaN,0.000000,0.000000,0.000000,0.000000
max,NaN,1.000000,1.000000,NaN,NaN,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000


In [44]:
# other predictors have max and min value 0 i.e all values are zero, they do not contribute in calculating the distance, so dropped 
df = df.drop(['usage_counter','normal_prio', 'vm_pgoff',\
              'hiwater_rss', 'nr_ptes', 'signal_nvcsw'], axis=1)

In [45]:
# checking for null values in data
df.isna().sum()

classification       0
prio                 0
static_prio          0
vm_truncate_count    0
map_count            0
total_vm             0
shared_vm            0
exec_vm              0
reserved_vm          0
nvcsw                0
nivcsw               0
os_Debian            0
os_Mac               0
os_Ubuntu            0
os_Windows           0
dtype: int64

In [46]:
from sklearn import preprocessing
y = df['classification'] # define label as nominal values
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y) # encode nominal labels to integers 

print(y_encoded)

df['classification'] = y_encoded
x_features = df.drop('classification',axis=1)

[1 1 1 ... 1 1 1]


In [47]:
df.head()

,classification,prio,static_prio,vm_truncate_count,map_count,total_vm,shared_vm,exec_vm,reserved_vm,nvcsw,nivcsw,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,1,0.18254,0.016007,0.199175,0.16651,0.052031,1.0,0.307692,0.249311,0.091519,0.0,0.0,0.0,0.0,0.0
1,1,0.18254,0.016007,0.199175,0.16651,0.052031,1.0,0.307692,0.249311,0.091519,0.0,0.0,0.0,0.0,1.0
2,1,0.18254,0.016007,0.199175,0.16651,0.052031,1.0,0.307692,0.249311,0.091519,0.0,0.0,1.0,0.0,0.0
3,1,0.18254,0.016007,0.199175,0.16651,0.052031,1.0,0.307692,0.249311,0.091519,0.0,0.0,0.0,1.0,0.0
4,1,0.18254,0.016007,0.199175,0.16651,0.052031,1.0,0.307692,0.249311,0.091519,0.0,0.0,1.0,0.0,0.0


In [48]:
df.head()

,classification,prio,static_prio,vm_truncate_count,map_count,total_vm,shared_vm,exec_vm,reserved_vm,nvcsw,nivcsw,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,1,0.18254,0.016007,0.199175,0.16651,0.052031,1.0,0.307692,0.249311,0.091519,0.0,0.0,0.0,0.0,0.0
1,1,0.18254,0.016007,0.199175,0.16651,0.052031,1.0,0.307692,0.249311,0.091519,0.0,0.0,0.0,0.0,1.0
2,1,0.18254,0.016007,0.199175,0.16651,0.052031,1.0,0.307692,0.249311,0.091519,0.0,0.0,1.0,0.0,0.0
3,1,0.18254,0.016007,0.199175,0.16651,0.052031,1.0,0.307692,0.249311,0.091519,0.0,0.0,0.0,1.0,0.0
4,1,0.18254,0.016007,0.199175,0.16651,0.052031,1.0,0.307692,0.249311,0.091519,0.0,0.0,1.0,0.0,0.0


In [49]:
# splitting the data into train and test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_features, y_encoded, 
                                                    test_size=0.2)

In [50]:
#out of curiosity I also tried hold out evaluation
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

# using both the metrics euclidean and manhattan distance
distances = ['euclidean','manhattan']
for i in distances: 
    print('metric: ', i)
    # k in range 5 to 105 with stepsize 10
    for k in range(5,105,10): 
        classifier=neighbors.KNeighborsClassifier(k, metric=i)
        classifier.fit(x_train, y_train)
        y_pred = classifier.predict(x_test)
        # Prints test accuracy, precision, recall and AUC
        print('K =', k, ', Accuracy: ', accuracy_score(y_test, y_pred), ', Precision: ', precision_score(y_test, y_pred, average='macro', zero_division=0),
            ', Recall: ', recall_score(y_test, y_pred, average='macro', zero_division=0),', AUC:', roc_auc_score(y_test, y_pred))

metric:  euclidean
K = 5 , Accuracy:  0.99985 , Precision:  0.9998498799039232 , Recall:  0.9998501648186995 , AUC: 0.9998501648186995
K = 15 , Accuracy:  0.9999 , Precision:  0.9999001298312193 , Recall:  0.9998998898788667 , AUC: 0.9998998898788667
K = 25 , Accuracy:  0.99895 , Precision:  0.9989498793279228 , Recall:  0.998950163729698 , AUC: 0.998950163729698
K = 35 , Accuracy:  0.9966 , Precision:  0.9966000599660005 , Recall:  0.9966006558867937 , AUC: 0.9966006558867936
K = 45 , Accuracy:  0.9952 , Precision:  0.995201204193457 , Recall:  0.995201204193457 , AUC: 0.9952012041934571
K = 55 , Accuracy:  0.9927 , Precision:  0.9927087801832006 , Recall:  0.9927026311701836 , AUC: 0.9927026311701836
K = 65 , Accuracy:  0.99125 , Precision:  0.9912600601609626 , Recall:  0.9912527944158813 , AUC: 0.9912527944158813
K = 75 , Accuracy:  0.98865 , Precision:  0.9886873693387144 , Recall:  0.9886551012726725 , AUC: 0.9886551012726725
K = 85 , Accuracy:  0.9852 , Precision:  0.98525099226

In [51]:
# 10-Fold cross validation on the dataset using euclidean distance
from sklearn.model_selection import cross_val_score

k_val = [3, 5, 11, 31, 51, 71, 91, 111, 131, 151, 171, 191, 211, 251, 271, 291]

for k in k_val: 
    classifier = neighbors.KNeighborsClassifier(k, metric='euclidean')
    accuracy = cross_val_score(classifier, x_features, y_encoded, cv=10, scoring='accuracy').mean()
    print('K =', k, ', Accuracy: ',accuracy)

K = 3 , Accuracy:  0.78889
K = 5 , Accuracy:  0.7859100000000001
K = 11 , Accuracy:  0.7809999999999999
K = 31 , Accuracy:  0.77349
K = 51 , Accuracy:  0.77732
K = 71 , Accuracy:  0.7784200000000001
K = 91 , Accuracy:  0.7798
K = 111 , Accuracy:  0.77699
K = 131 , Accuracy:  0.77518
K = 151 , Accuracy:  0.76793
K = 171 , Accuracy:  0.76773
K = 191 , Accuracy:  0.7670699999999999
K = 211 , Accuracy:  0.7632599999999999
K = 251 , Accuracy:  0.75099
K = 271 , Accuracy:  0.7526400000000001
K = 291 , Accuracy:  0.7588999999999999


In [52]:
# 10-Fold cross validation on the dataset using manhattan distance
k_val = [3, 5, 11, 31, 51, 71, 91, 111, 131, 151, 171, 191, 211, 251, 271, 291]

for k in k_val: 
    classifier = neighbors.KNeighborsClassifier(k, metric='manhattan')
    accuracy = cross_val_score(classifier, x_features, y_encoded, cv=10, scoring='accuracy').mean()
    print('K =', k, ', Accuracy: ',accuracy)

K = 3 , Accuracy:  0.8068500000000001
K = 5 , Accuracy:  0.80443
K = 11 , Accuracy:  0.7999099999999999
K = 31 , Accuracy:  0.80442
K = 51 , Accuracy:  0.8047599999999999
K = 71 , Accuracy:  0.80351
K = 91 , Accuracy:  0.80593
K = 111 , Accuracy:  0.80916
K = 131 , Accuracy:  0.8032
K = 151 , Accuracy:  0.8051999999999999
K = 171 , Accuracy:  0.80586
K = 191 , Accuracy:  0.80528
K = 211 , Accuracy:  0.79811
K = 251 , Accuracy:  0.79233
K = 271 , Accuracy:  0.7952
K = 291 , Accuracy:  0.79949


In [53]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, precision_score, recall_score
from sklearn import neighbors

# Assuming x_features and y_encoded are your feature and target variables

k_val = [3, 5, 11, 31, 51, 71, 91, 111, 131, 151, 171, 191, 211, 251, 271, 291]

for k in k_val: 
    classifier = neighbors.KNeighborsClassifier(k, metric='euclidean')

    # Calculate accuracy
    accuracy = cross_val_score(classifier, x_features, y_encoded, cv=10, scoring='accuracy').mean()

    # Calculate precision
    precision = cross_val_score(classifier, x_features, y_encoded, cv=10, scoring=make_scorer(precision_score)).mean()

    # Calculate recall
    recall = cross_val_score(classifier, x_features, y_encoded, cv=10, scoring=make_scorer(recall_score)).mean()

    print('K =', k, ', Accuracy:', accuracy, ', Precision:', precision, ', Recall:', recall)

K = 3 , Accuracy: 0.78889 , Precision: 0.7919387867242302 , Recall: 0.8069599999999999
K = 5 , Accuracy: 0.7859100000000001 , Precision: 0.7923652881380647 , Recall: 0.7990200000000001
K = 11 , Accuracy: 0.7809999999999999 , Precision: 0.7909925332463366 , Recall: 0.7882199999999999
K = 31 , Accuracy: 0.77349 , Precision: 0.7783442591584715 , Recall: 0.7860199999999999
K = 51 , Accuracy: 0.77732 , Precision: 0.789142914527225 , Recall: 0.7863
K = 71 , Accuracy: 0.7784200000000001 , Precision: 0.7934212935475051 , Recall: 0.7827399999999999
K = 91 , Accuracy: 0.7798 , Precision: 0.7995343447953918 , Recall: 0.7776599999999999
K = 111 , Accuracy: 0.77699 , Precision: 0.7980708668900063 , Recall: 0.77672
K = 131 , Accuracy: 0.77518 , Precision: 0.7951990342762876 , Recall: 0.7774599999999999
K = 151 , Accuracy: 0.76793 , Precision: 0.7840193019598036 , Recall: 0.7761
K = 171 , Accuracy: 0.76773 , Precision: 0.7834847144040152 , Recall: 0.7787200000000001
K = 191 , Accuracy: 0.767069999999

In [54]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier

# Assuming x_features and y_encoded are your feature and target variables

k_val = [3, 5, 11, 31, 51, 71, 91, 111, 131, 151, 171, 191, 211, 251, 271, 291]

for k in k_val:
    classifier = KNeighborsClassifier(k, metric='manhattan',weights='distance')
#Hyper-Parameters changed - Distance 
    # Calculate accuracy
    accuracy = cross_val_score(classifier, x_features, y_encoded, cv=10, scoring='accuracy').mean()

    # Calculate precision
    precision = cross_val_score(classifier, x_features, y_encoded, cv=10, scoring=make_scorer(precision_score)).mean()

    # Calculate recall
    recall = cross_val_score(classifier, x_features, y_encoded, cv=10, scoring=make_scorer(recall_score)).mean()

    print('K =', k, ', Accuracy:', accuracy, ', Precision:', precision, ', Recall:', recall)

K = 3 , Accuracy: 0.8068500000000001 , Precision: 0.8108903070068443 , Recall: 0.81932
K = 5 , Accuracy: 0.80468 , Precision: 0.8098712379509271 , Recall: 0.8141400000000001
K = 11 , Accuracy: 0.8002499999999999 , Precision: 0.808418707712533 , Recall: 0.80282
K = 31 , Accuracy: 0.80493 , Precision: 0.813872612997249 , Recall: 0.80754
K = 51 , Accuracy: 0.80578 , Precision: 0.8148582210695465 , Recall: 0.81182
K = 71 , Accuracy: 0.8049000000000002 , Precision: 0.811230972699241 , Recall: 0.81368
K = 91 , Accuracy: 0.80758 , Precision: 0.8144153797021488 , Recall: 0.8168399999999998
K = 111 , Accuracy: 0.8103 , Precision: 0.8170783864180245 , Recall: 0.82358
K = 131 , Accuracy: 0.8063499999999999 , Precision: 0.8119991271666948 , Recall: 0.8263999999999999
K = 151 , Accuracy: 0.8059200000000001 , Precision: 0.8100704597521051 , Recall: 0.8303
K = 171 , Accuracy: 0.8091900000000001 , Precision: 0.8111625696217388 , Recall: 0.83754
K = 191 , Accuracy: 0.8075800000000001 , Precision: 0.808